### Этот ноутбук содержит программу для сбора треугольника из файла IBNR Best Test

In [ ]:
import os

In [ ]:
import pandas as pd
import datetime as dt
import numpy as np 
from openpyxl import load_workbook

In [ ]:
pd.options.mode.chained_assignment = None

In [ ]:
#функция для перевода столбика в единый формат времени
def to_datetime(x):
    for date in x:
        if str(type(date)) != "<class 'datetime.datetime'>" and pd.isna(date) == False and str(type(date)) != "<class 'pandas._libs.tslibs.timestamps.Timestamp'>":
            x = x.replace(date, dt.datetime.fromordinal(693594+int(date)))
    x = pd.to_datetime(x, errors='coerce')
    return x

In [ ]:
folderpath = r'' # cюда вставить название папки, где будут все файлы для треугольников и только они
filepaths =  [os.path.join(folderpath, name) for name in os.listdir(folderpath)] #этой командой создаем список всех адресов 'filepaths16', содержихся по адресу folderpath 
outputpath = r'' #адрес, куда выводить аутпут

In [ ]:
last_quartal='2021_3'
SheetName='Выгрузка' #сделать по всем листам
Sum=15 #все в питоновских цифрах, т.е. (номер - 1)
Event_date=10 
Pay_date=16
LoB_given=30
first_row =7 #первая строка листа с информацией в питоновских цифрах

Dates_for_comparison = 16
Sum_for_comparison = 15
Condition_for_comparison = '0' #'0', если доп условия нет. Само условие
Condition_cell_for_comparison = 'Год_Квартал оплаты' #название колонки таблицы Paid_losses_1, для которой делаем условие для comparison table


Cell_X = 23 #Клетка для дополнительного условия Х
Condition_for_cell_X = 'да' # если условия Х нет, то в этом парметре поставить '0'
Cell_Y = 34 #Клетка для дополнительного условия Y
Condition_for_cell_Y = '0'#'Возмещение ущерба имуществу'#'Возмещение ущерба имуществу' #'Возмещение ущерба жизни/здоровью потерпевших в ДТП' # если условия Y нет, то в этом парметре поставить '0'
Cell_Z = 26 #Клетка для дополнительного условия Z
Condition_for_cell_Z = '0'#'да'#'нет'#'да' # если условия Z нет, то в этом парметре поставить '0'
Cell_Delta = 34 #Клетка для дополнительного условия Z
Condition_for_cell_Delta = '0' # если условия Z нет, то в этом парметре поставить '0'
Cell_Omicron = 34 #Клетка для дополнительного условия Z
Condition_for_cell_Omicron = '0' # если условия Z нет, то в этом парметре поставить '0'


In [ ]:
folderpath[-20:]

In [ ]:
conditions_initial = [Condition_for_cell_X,Condition_for_cell_Y,Condition_for_cell_Z,Condition_for_cell_Delta,Condition_for_cell_Omicron]
conditions = []
for i in conditions_initial:
    if i != '0':
        conditions.append(i)
cells = [Cell_X, Cell_Y, Cell_Z, Cell_Delta, Cell_Omicron]        
cells = cells[:len(conditions)]
print(conditions) 
print(cells)        

print('--------')

for path in filepaths:
    Paid_losses = pd.read_excel(path, sheet_name=SheetName, skiprows=list(range(0,first_row)), header=None)
    
    #данные для comparison
    Sum_for_comparison_ = pd.Series(Paid_losses[Sum_for_comparison], name = 'Sum_for_comparison')
    Dates_for_comparison_ = pd.Series(Paid_losses[Dates_for_comparison], name = 'Dates_for_comparison')

    #список данных учетных групп
    LoBs = sorted([item for item in list(set(list(Paid_losses[LoB_given]))) if pd.isnull(item) == False])
    
    output_name = outputpath+'treug'+path[:-20]
    writer = pd.ExcelWriter(output_name, engine='xlsxwriter')
    
    comparison_table = pd.DataFrame(columns=['Дата']+LoBs)
    ages = list(range(2000, int(last_quartal[:4])+1))

    quartiles_ = []
    for age in ages:
        quartiles_.append(str(age)+'_1')
        quartiles_.append(str(age)+'_2')
        quartiles_.append(str(age)+'_3')
        quartiles_.append(str(age)+'_4')

    comparison_table['Дата'] = quartiles_
    comparison_table = comparison_table.set_index('Дата').fillna(0)
    display(comparison_table)

    for LoB in LoBs:

        print(LoB)
        Paid_losses_LOB = Paid_losses.loc[Paid_losses[LoB_given] == LoB] #таблица со строками только для первой группы
        Paid_losses_1 = Paid_losses_LOB[[Sum, Event_date, Pay_date, LoB_given]] #оставили только нужные колонки
        Paid_losses_1[cells]=Paid_losses_LOB.loc[:,cells]
        Paid_losses_1['Sum_for_comparison'] = Sum_for_comparison_
        Paid_losses_1['Dates_for_comparison'] = Dates_for_comparison_
        for i in range(len(cells)):
            Paid_losses_1 = Paid_losses_1.loc[((Paid_losses_1[cells[i]] == conditions[i]))]

        display(Paid_losses_1)

        #теперь сделаю колонку, в которой будет количесвто дней, прошедших со случая до оплаты, может быть полезно
        Paid_losses_1['Time Delta'] = to_datetime(Paid_losses_1[Pay_date]) - to_datetime(Paid_losses_1[Event_date])

        #также добавим колонки "Квартал случая" и "Квартал оплаты", а также года и год_квартал
        Paid_losses_1['Квартал случая'] = np.ceil(to_datetime(Paid_losses_1[Event_date]).dt.month / 3)
        Paid_losses_1['Год случая'] = to_datetime(Paid_losses_1[Event_date]).dt.year
        Paid_losses_1['Квартал оплаты'] = np.ceil(to_datetime(Paid_losses_1[Pay_date]).dt.month / 3)
        Paid_losses_1['Год оплаты'] = to_datetime(Paid_losses_1[Pay_date]).dt.year
        Paid_losses_1['Квартал сравнения'] = np.ceil(to_datetime(Paid_losses_1[Dates_for_comparison]).dt.month / 3)
        Paid_losses_1['Год сравнения'] = to_datetime(Paid_losses_1[Dates_for_comparison]).dt.year


        god_kvartal_case = []
        for i in range(len(Paid_losses_1)):
            god_kvartal_case.append(str(Paid_losses_1['Год случая'].iloc[i])[:4] + '_' + str(Paid_losses_1['Квартал случая'].iloc[i])[0])
        Paid_losses_1['Год_Квартал случая'] = god_kvartal_case

        god_kvartal_pay = []
        for i in range(len(Paid_losses_1)):
            god_kvartal_pay.append(str(Paid_losses_1['Год оплаты'].iloc[i])[:4] + '_' + str(Paid_losses_1['Квартал оплаты'].iloc[i])[0])
        Paid_losses_1['Год_Квартал оплаты'] = god_kvartal_pay

        god_kvartal_comparison = []
        for i in range(len(Paid_losses_1)):
            god_kvartal_comparison.append(str(Paid_losses_1['Год сравнения'].iloc[i])[:4] + '_' + str(Paid_losses_1['Квартал сравнения'].iloc[i])[0])
        Paid_losses_1['Год_Квартал сравнения'] = god_kvartal_pay


        #И финально добавляем колонку "Количество кварталов от оплаты до случая"
        Paid_losses_1['Количество кварталов от оплаты до случая'] = 4 * (Paid_losses_1['Год оплаты'] - Paid_losses_1['Год случая']) + Paid_losses_1['Квартал оплаты'] - Paid_losses_1['Квартал случая']

        display(Paid_losses_1)

        #Теперь делаем треугольники
        triangle = pd.DataFrame(columns=['квартал'] + list(range(71))) #макет с колонками 0 - 70
        years = []
        for j in range(sorted(list(set(Paid_losses_1['Год случая'].to_list())))[0], int(last_quartal[:4])+1):
            years.append(j)

        if len(years)>0: #!!!!!!
            year = years[0]
            quartiles = []
            while year < years[len(years)-1]:
                quartiles.append(str(year)+'_1')
                quartiles.append(str(year)+'_2')
                quartiles.append(str(year)+'_3')
                quartiles.append(str(year)+'_4')
                year+=1

            quartal = 1
            while quartal <= int(last_quartal[len(last_quartal)-1]):
                quartiles.append(str(year)+'_'+str(quartal))
                quartal += 1

            triangle['квартал']=quartiles
            triangle = triangle.set_index('квартал')
            for i in range(len(triangle)):
                if i<=70:
                    triangle.iloc[:len(triangle)-i, i] = triangle.iloc[:len(triangle)-i, i].fillna(0)

            #теперь раскидываем значения из таблицы по треугольнику и таблице сверки:
            for i in range(len(Paid_losses_1)):
                if Paid_losses_1['Количество кварталов от оплаты до случая'].iloc[i] >=0 and pd.isna(Paid_losses_1[Sum].iloc[i]) == False:
                        triangle.loc[Paid_losses_1['Год_Квартал случая'].iloc[i], Paid_losses_1['Количество кварталов от оплаты до случая'].iloc[i]] += Paid_losses_1[Sum].iloc[i]
                        if Condition_for_comparison != '0':
                            if Paid_losses_1[Condition_cell_for_comparison].iloc[i] == Condition_for_comparison:
                                comparison_table.loc[Paid_losses_1['Год_Квартал сравнения'].iloc[i], Paid_losses_1[LoB_given].iloc[i]] += Paid_losses_1['Sum_for_comparison'].iloc[i]
                        elif Condition_for_comparison == '0':
                            comparison_table.loc[Paid_losses_1['Год_Квартал сравнения'].iloc[i], Paid_losses_1[LoB_given].iloc[i]] += Paid_losses_1['Sum_for_comparison'].iloc[i]


            triangle_incr = triangle.copy()

            #теперь делаем треуголоьник кумулятивным
            print(triangle)
            for i in range(1, 70):
                triangle[[i]] = pd.DataFrame(triangle[[i-1,i]].sum(axis=1, skipna=False))         

            #и записываем его в указанный файл
            triangle.to_excel(writer, sheet_name='кум_УГ_'+str(LoB))
            triangle_incr.to_excel(writer, sheet_name='инкр_УГ_'+str(LoB))

        else:
            print('No data for LoB', LoB)

        #Paid_losses_1.to_excel(writer, sheet_name='Paid_losses_УГ_'+str(LoB))


    comparison_table.to_excel(writer, sheet_name='COMPARISON_TABLE')
    writer.save()